In [1]:
import numba
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import convolve, generate_binary_structure
import matplotlib
matplotlib.use('QT5Agg')
%matplotlib qt

In [2]:
class isingmodel:
    def __init__(self, N, J=1, B=0):
        self.J = J
        self.N = N
        self.init_random = np.random.random((N, N))
        self.grid_n = np.ones((N, N))
        # self.grid_n[self.init_random >= .5] = 1
        # self.grid_n[self.init_random < .5] = -1
    
    def get_energy(self, grid):
        kernel = generate_binary_structure(2, 1)
        kernel[1][1] = False
        result = - self.J * grid * convolve(grid, kernel, mode='wrap')
        return result.sum()
    
    def grid(self):
        return self.grid_n
    
    @staticmethod
    @numba.njit("Tuple((f8[:], f8[:], f8[:,:]))(f8[:,:], i8, f8, f8, i8)", nopython=True, nogil=True)
    def metropolis(arr_spin, times, beta, energy, N):
        arr_spin = arr_spin.copy()
        total_spin = np.zeros(times - 1)
        total_energy = np.zeros(times - 1)
        for t in range(0, times - 1):
            x = np.random.randint(0,N)
            y = np.random.randint(0,N)
            spin_t = arr_spin[x, y]
            spin_prime = -spin_t
            E_t = 0
            E_prime = 0
            neighbours = [(x - 1) % N, (x + 1) % N, (y - 1) % N, (y + 1) % N]
            for nx in [neighbours[0], neighbours[1]]:
                E_t += - spin_t * arr_spin[nx, y]
                E_prime += - spin_prime * arr_spin[nx, y]
            for ny in [neighbours[2], neighbours[3]]:
                E_t += - spin_t * arr_spin[x, ny]
                E_prime += - spin_prime * arr_spin[x, ny]
            dE = E_prime - E_t
            if (dE > 0) & (np.random.random() < np.exp(-beta * dE)):
                arr_spin[x, y] = spin_prime
                energy += dE
            elif dE <= 0:
                arr_spin[x, y] = spin_prime
                energy += dE
            total_spin[t] = arr_spin.sum()
            total_energy[t] = energy
            
        return total_spin, total_energy, arr_spin
        
    def plot(self, cmap, times, beta, save=False):
        fig, axes = plt.subplots(dpi=150)
        plt.rcParams["font.family"] = "times"
        plt.rcParams["text.usetex"] = True
        plt.title(fr'{self.N}$\times${self.N}-lattice after {times:.1e} thermalization steps' + '\n' + rf'$\beta={beta}$', fontsize=12, pad=10)
        plt.imshow(equilibrium, cmap=cmap)
        if save:
            fig.savefig(f"/Users/danielmiksch/Downloads/{self.N}by{self.N}_grid.pdf")
            
    def magnetization(self, spins):
        return spins / self.N ** 2
    
    def print_energy(self):
        print(self.get_energy(self.grid_n))

/Users/danielmiksch/miniconda3/envs/juplab/lib/python3.10/site-packages/numba/core/decorators.py:282: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


# a)
$50\times 50$-grid after $10\,000\cdot 50^2$ thermalization updates at $\beta=0.45$. The lattice is being initialized to the ferromagnetic ground state, i.e. all spins are equal to one.

In [3]:
N = 50
steps = 25e6
beta = 0.45
model = isingmodel(N=N)
spin_grid = model.grid()
energy = model.get_energy(spin_grid)

In [4]:
spins, energies, equilibrium = model.metropolis(spin_grid, steps, beta, energy, N)

In [5]:
model.plot(cmap='binary', times=steps, beta=beta, save=False)

In [6]:
spins[-25000:].mean()

2040.40544

# b)


In [10]:
betas = np.linspace(0.3, 0.7, 3)
N = [20]
sweeps = [10 * i**2 for i in N]
sweeps

[4000]

In [18]:
model = isingmodel(N=20)
spin_grid = model.grid()
energy = model.get_energy(spin_grid)
spins, s, t = model.metropolis(spin_grid, steps, 0.3, energy, 20)
spins[-4000:].mean() / 20**2

0.07030625

In [19]:
def get_magnetization(N, betas, sweeps):
    magnetization_list = np.zeros((len(N), len(betas)))
    for index1, i in enumerate(N):
        magnetization = np.zeros(len(betas))
        for index2, s in enumerate(betas):
            model = isingmodel(N=i)
            spin_grid = model.grid()
            energy = model.get_energy(spin_grid)
            spins, energies, equilibrium = model.metropolis(spin_grid, steps, s, energy, i)
            magnetization[index2] = spins[-sweeps[index1]:].mean() / i**2
        magnetization_list[index1] = magnetization
            
    return magnetization_list

In [20]:
magnetization = get_magnetization(N, betas, sweeps)
magnetization

array([[0.0284775 , 0.92733   , 0.98846875]])

In [ ]:
# fig, axes = plt.subplots(1, 2, figsize=(12,4))
# 
# plt.rcParams["font.family"] = "times"
# plt.rcParams["text.usetex"] = True
# 
# ax = axes[0]
# ax.plot(spins/50**2)
# ax.set_xlabel('Algorithm Time Steps')
# ax.set_ylabel(r'Magnetization $\langle M\rangle$')
# ax.grid()
# ax = axes[1]
# ax.plot(energies)
# ax.set_xlabel('Algorithm Time Steps')
# ax.set_ylabel(r'Energy $E$')
# ax.grid()
# fig.tight_layout()
# fig.suptitle(r'Evolution of Average Spin and Energy', y=1.07, size=18)
# plt.show()